In [ ]:
# %% [markdown]
# # 供应商选择与生产优化：AHP + 线性规划综合案例
# 
# 本案例展示如何结合使用层次分析法(AHP)和线性规划解决企业决策问题：
# 1. **AHP部分**：选择最佳零部件供应商（3家候选）
# 2. **线性规划部分**：基于选定供应商制定最优生产计划
# 
# **业务背景**：某家电企业需要采购核心零部件并制定下季度生产计划

# %%
# 导入必要的库
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 设置全局样式
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
plt.style.use('ggplot')

In [ ]:
# %% [markdown]
# ## 第一部分：层次分析法(AHP)选择最佳供应商

# %% [markdown]
# ### 1. 问题定义
# - **目标**：选择最佳零部件供应商
# - **候选供应商**：A公司、B公司、C公司
# - **评估维度**：价格、质量、交货稳定性、技术支持

# %%
# %% [markdown]
# ### 2. 建立递阶层次结构
# ```
# 目标层：选择最佳供应商
# │
# ├── 准则层：价格
# ├── 准则层：质量
# ├── 准则层：交货稳定性
# └── 准则层：技术支持
#     │
#     ├── 方案层：A公司
#     ├── 方案层：B公司
#     └── 方案层：C公司
# ```

# %%
# %% [markdown]
# ### 3. 准则层判断矩阵与权重计算

# %%
# 准则层判断矩阵（价格、质量、交货稳定性、技术支持）
criteria_matrix = np.array([
    [1, 3, 5, 7],
    [1/3, 1, 3, 5],
    [1/5, 1/3, 1, 3],
    [1/7, 1/5, 1/3, 1]
])

# 创建准则层判断矩阵的DataFrame用于展示
criteria_names = ['价格', '质量', '交货稳定性', '技术支持']
criteria_df = pd.DataFrame(criteria_matrix, 
                          index=criteria_names, 
                          columns=criteria_names)

display(Markdown("#### 准则层判断矩阵（管理层打分）"))
display(criteria_df.style.format("{:.2f}").background_gradient(cmap='Blues'))

# %%
def ahp_weights(matrix):
    """
    计算AHP判断矩阵的权重和进行一致性检验
    
    参数:
    matrix -- 判断矩阵 (numpy array)
    
    返回:
    weights -- 权重向量
    ci -- 一致性指标
    cr -- 一致性比率
    """
    # 1. 列归一化
    column_sums = matrix.sum(axis=0)
    normalized = matrix / column_sums
    
    # 2. 行求和并归一化得到权重
    weights = normalized.sum(axis=1) / matrix.shape[0]
    
    # 3. 计算最大特征值 λ_max
    Aw = matrix @ weights
    lambda_max = np.mean(Aw / weights)
    
    # 4. 一致性指标 CI
    n = matrix.shape[0]
    ci = (lambda_max - n) / (n - 1)
    
    # 5. 随机一致性指标 RI (查表)
    ri_values = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45}
    ri = ri_values.get(n, 0)
    
    # 6. 一致性比率 CR
    cr = ci / ri if ri != 0 else 0
    
    return weights, ci, cr

# 计算准则层权重
criteria_weights, criteria_ci, criteria_cr = ahp_weights(criteria_matrix)

# 展示结果
display(Markdown("#### 准则层权重计算结果"))
print(f"最大特征值 λ_max = {criteria_ci * (4-1) + 4:.3f}")
print(f"一致性指标 CI = {criteria_ci:.4f}")
print(f"一致性比率 CR = {criteria_cr:.4f} {'< 0.1，通过一致性检验！' if criteria_cr < 0.1 else '> 0.1，需调整判断矩阵！'}")

# 创建权重DataFrame
weights_df = pd.DataFrame({
    '准则': criteria_names,
    '权重': criteria_weights,
    '占比': [f"{w:.1%}" for w in criteria_weights]
})
display(weights_df.set_index('准则').style.format({'权重': "{:.3f}"}).bar(subset=['权重'], color='#5fba7d'))

In [ ]:
# %%
# %% [markdown]
# ### 4. 方案层判断矩阵与权重计算

# %%
# 价格维度判断矩阵（基于实际价格数据：A=120元, B=105元, C=135元）
price_matrix = np.array([
    [1, 120/105, 120/135],
    [105/120, 1, 105/135],
    [135/120, 135/105, 1]
])

# 质量维度判断矩阵（基于次品率：A=0.8%, B=0.5%, C=1.2%）
# 注意：次品率越低越好，所以需要取倒数
quality_matrix = np.array([
    [1, 0.5/0.8, 0.5/1.2],
    [0.8/0.5, 1, 0.8/1.2],
    [1.2/0.5, 1.2/0.8, 1]
])

# 交货稳定性维度判断矩阵（基于历史数据评分：A=85, B=90, C=80）
delivery_matrix = np.array([
    [1, 90/85, 90/80],
    [85/90, 1, 85/80],
    [80/90, 80/85, 1]
])

# 技术支持维度判断矩阵（基于服务评分：A=8, B=7, C=9）
support_matrix = np.array([
    [1, 7/8, 7/9],
    [8/7, 1, 8/9],
    [9/7, 9/8, 1]
])

# 所有方案层判断矩阵
all_matrices = {
    '价格': price_matrix,
    '质量': quality_matrix,
    '交货稳定性': delivery_matrix,
    '技术支持': support_matrix
}

# 供应商名称
suppliers = ['A公司', 'B公司', 'C公司']

# 计算每个准则下的供应商权重
supplier_weights = {}
for name, matrix in all_matrices.items():
    weights, ci, cr = ahp_weights(matrix)
    supplier_weights[name] = weights
    
    # 展示每个判断矩阵的结果
    display(Markdown(f"#### {name}维度判断矩阵"))
    
    # 创建判断矩阵DataFrame
    matrix_df = pd.DataFrame(matrix, index=suppliers, columns=suppliers)
    display(matrix_df.style.format("{:.2f}").background_gradient(cmap='Oranges'))
    
    print(f"一致性比率 CR = {cr:.4f} {'< 0.1，通过一致性检验！' if cr < 0.1 else '> 0.1，需调整判断矩阵！'}")
    
    # 创建权重DataFrame
    weights_df = pd.DataFrame({
        '供应商': suppliers,
        '权重': weights,
        '占比': [f"{w:.1%}" for w in weights]
    })
    display(weights_df.set_index('供应商').style.format({'权重': "{:.3f}"}).bar(subset=['权重'], color='#d65f5f'))
    print("\n" + "-"*80 + "\n")

In [ ]:
# %%
# %% [markdown]
# ### 5. 综合权重计算与最终决策

# %%
# 创建综合权重表格
comprehensive_weights = []
for supplier in suppliers:
    # 获取该供应商在各准则下的权重
    supplier_idx = suppliers.index(supplier)
    weights = [supplier_weights[criteria][supplier_idx] for criteria in criteria_names]
    
    # 计算综合得分
    score = sum(w * criteria_weights[i] for i, w in enumerate(weights))
    
    comprehensive_weights.append({
        '供应商': supplier,
        '价格权重': weights[0],
        '质量权重': weights[1],
        '交货稳定性权重': weights[2],
        '技术支持权重': weights[3],
        '综合得分': score
    })

# 转换为DataFrame并排序
results_df = pd.DataFrame(comprehensive_weights)
results_df = results_df.sort_values('综合得分', ascending=False)

# 计算占比
results_df['综合占比'] = results_df['综合得分'] / results_df['综合得分'].sum()

# 展示结果
display(Markdown("## 供应商综合评估结果"))
display(results_df.set_index('供应商').style.format({
    '价格权重': "{:.3f}",
    '质量权重': "{:.3f}",
    '交货稳定性权重': "{:.3f}",
    '技术支持权重': "{:.3f}",
    '综合得分': "{:.3f}",
    '综合占比': "{:.1%}"
}).background_gradient(subset=['综合得分'], cmap='viridis'))

# 可视化综合得分
plt.figure(figsize=(10, 6))
bars = plt.bar(results_df['供应商'], results_df['综合得分'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('供应商综合评估得分对比', fontsize=16)
plt.ylabel('综合得分', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 在柱子上显示具体数值
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height:.3f}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  # 3点垂直偏移
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=12)

plt.tight_layout()
plt.savefig('supplier_comparison.png', dpi=300)
plt.show()

# %%
# %% [markdown]
# ### 6. AHP决策结论

# %%
# 选择得分最高的供应商
best_supplier = results_df.iloc[0]['供应商']
best_score = results_df.iloc[0]['综合得分']
second_best = results_df.iloc[1]['供应商']
second_score = results_df.iloc[1]['综合得分']

display(Markdown(f"## ✅ AHP决策结论"))
display(Markdown(f"**最佳供应商：{best_supplier}** (综合得分: {best_score:.3f})"))
display(Markdown(f"**次优供应商：{second_best}** (综合得分: {second_score:.3f})"))
display(Markdown(f"\n**优势分析**：{best_supplier}在关键指标上表现突出，特别是在**{criteria_names[np.argmax(criteria_weights)]}**方面（权重: {max(criteria_weights):.1%}），这是企业最重视的因素。"))

In [ ]:
# %% [markdown]
# ## 第二部分：线性规划制定生产计划

# %% [markdown]
# ### 1. 问题背景
# 选定{best_supplier}后，我们需要制定下季度生产计划：
# - 生产两种产品：智能空调(产品X)、智能冰箱(产品Y)
# - **目标**：最大化总利润
# - **约束条件**：
#   - 原材料供应：{best_supplier}每月最多提供400单位零部件
#   - 生产能力：总工时不超过600小时
#   - 市场需求：X至少生产50台，Y至少生产40台
#   - 库存限制：X最多生产200台，Y最多生产150台

# %%
# 产品数据
products = ['智能空调(X)', '智能冰箱(Y)']
profit = [800, 1200]  # 单位利润(元)
material_usage = [2, 3]  # 零部件消耗
labor_usage = [3, 4]     # 工时消耗

# 创建产品数据表格
product_data = pd.DataFrame({
    '指标': ['单位利润(元)', '零部件消耗', '工时消耗'],
    '智能空调(X)': [profit[0], material_usage[0], labor_usage[0]],
    '智能冰箱(Y)': [profit[1], material_usage[1], labor_usage[1]]
})

display(Markdown("### 产品数据汇总"))
display(product_data.set_index('指标').style.format("{:.0f}"))

# %%
# %% [markdown]
# ### 2. 线性规划模型

# %%
# 约束条件
material_constraint = 400    # 原材料限制
labor_constraint = 600       # 工时限制
min_x = 50                   # X最低需求
min_y = 40                   # Y最低需求
max_x = 200                  # X最高产能
max_y = 150                  # Y最高产能

display(Markdown("### 约束条件汇总"))
constraints_df = pd.DataFrame({
    '约束类型': ['原材料供应', '生产能力', 'X最低需求', 'Y最低需求', 'X最高产能', 'Y最高产能'],
    '约束值': [material_constraint, labor_constraint, min_x, min_y, max_x, max_y],
    '数学表达式': [
        f'{material_usage[0]}X + {material_usage[1]}Y ≤ {material_constraint}',
        f'{labor_usage[0]}X + {labor_usage[1]}Y ≤ {labor_constraint}',
        f'X ≥ {min_x}',
        f'Y ≥ {min_y}',
        f'X ≤ {max_x}',
        f'Y ≤ {max_y}'
    ]
})
display(constraints_df.set_index('约束类型'))

# %%
# %% [markdown]
# ### 3. Python实现线性规划求解

# %%
# 目标函数系数（最小化问题，取负）
c = [-p for p in profit]  # 对应 -800x -1200y

# 不等式约束：Ax <= b
A = [
    material_usage,       # 原材料约束
    labor_usage,          # 工时约束
    [-1, 0],              # x >= min_x → -x <= -min_x
    [0, -1],              # y >= min_y → -y <= -min_y
    [1, 0],               # x <= max_x
    [0, 1]                # y <= max_y
]
b = [material_constraint, labor_constraint, -min_x, -min_y, max_x, max_y]

# 求解线性规划
res = linprog(
    c, 
    A_ub=A,
    b_ub=b
)

# 结果处理
x_opt, y_opt = res.x
max_profit = -res.fun  # 还原为最大利润

In [ ]:
# %%
# %% [markdown]
# ### 4. 结果分析与可视化

# %%
display(Markdown("## 线性规划求解结果"))
print(f"✅ 最优生产计划: 智能空调(X) = {x_opt:.0f}台, 智能冰箱(Y) = {y_opt:.0f}台")
print(f"💰 最大化季度利润: {max_profit:,.0f}元")

print("\n📊 约束条件验证:")
print(f"• 原材料使用: {material_usage[0]}×{x_opt:.0f} + {material_usage[1]}×{y_opt:.0f} = {material_usage[0]*x_opt+material_usage[1]*y_opt:.0f} ≤ {material_constraint}")
print(f"• 工时使用: {labor_usage[0]}×{x_opt:.0f} + {labor_usage[1]}×{y_opt:.0f} = {labor_usage[0]*x_opt+labor_usage[1]*y_opt:.0f} ≤ {labor_constraint}")
print(f"• 市场需求: X={x_opt:.0f} ≥ {min_x}, Y={y_opt:.0f} ≥ {min_y}")
print(f"• 产能限制: X={x_opt:.0f} ≤ {max_x}, Y={y_opt:.0f} ≤ {max_y}")

# %%
# 可视化可行域
x = np.linspace(0, max_x, 1000)

plt.figure(figsize=(12, 8))

# 绘制约束边界
plt.plot(x, (material_constraint - material_usage[0]*x) / material_usage[1], 'r-', label=f'原材料: {material_usage[0]}x+{material_usage[1]}y≤{material_constraint}')
plt.plot(x, (labor_constraint - labor_usage[0]*x) / labor_usage[1], 'b-', label=f'工时: {labor_usage[0]}x+{labor_usage[1]}y≤{labor_constraint}')
plt.axvline(x=min_x, color='g', linestyle='--', label=f'X≥{min_x}')
plt.axhline(y=min_y, color='m', linestyle='--', label=f'Y≥{min_y}')
plt.axvline(x=max_x, color='c', linestyle='--', label=f'X≤{max_x}')
plt.axhline(y=max_y, color='y', linestyle='--', label=f'Y≤{max_y}')

# 计算可行域
y1 = np.minimum((material_constraint - material_usage[0]*x) / material_usage[1], 
                (labor_constraint - labor_usage[0]*x) / labor_usage[1])
y2 = np.maximum(min_y, np.zeros_like(x))
valid_mask = (x >= min_x) & (x <= max_x) & (y1 >= min_y) & (y1 <= max_y)

# 填充可行域
plt.fill_between(x, y2, y1, where=valid_mask, color='gray', alpha=0.2, label='可行域')

# 标记最优解
plt.plot(x_opt, y_opt, 'ko', markersize=10, label=f'最优解 ({int(x_opt)}, {int(y_opt)})')

# 标记角点（可选）
corner_points = [
    (min_x, min_y),
    (min_x, min(max_y, (material_constraint - material_usage[0]*min_x)/material_usage[1], (labor_constraint - labor_usage[0]*min_x)/labor_usage[1])),
    (min(max_x, (material_constraint - material_usage[1]*min_y)/material_usage[0], (labor_constraint - labor_usage[1]*min_y)/labor_usage[0]), min_y),
    (x_opt, y_opt)
]

for point in corner_points:
    plt.plot(point[0], point[1], 'ro', markersize=6)

# 添加目标函数等高线（可选）
profit_levels = np.linspace(0, max_profit, 5)
for p in profit_levels:
    plt.plot(x, (p - profit[0]*x) / profit[1], 'k--', alpha=0.3)

plt.xlim(0, max_x + 10)
plt.ylim(0, max_y + 10)
plt.xlabel('智能空调产量 (X)', fontsize=12)
plt.ylabel('智能冰箱产量 (Y)', fontsize=12)
plt.title('生产计划可行域与最优解', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('production_planning.png', dpi=300)
plt.show()

In [ ]:
# %%
# %% [markdown]
# ## 第三部分：综合决策建议

# %%
# %% [markdown]
# ### 1. 供应商选择建议

# %%
display(Markdown("## 📌 综合决策建议"))

# 供应商选择建议
display(Markdown(f"### ✅ 供应商选择建议"))
display(Markdown(f"**推荐选择：{best_supplier}**"))
display(Markdown(f"- **综合得分**：{best_score:.3f}（领先第二名 {best_score - second_score:.3f} 分）"))
display(Markdown(f"- **核心优势**：在**{criteria_names[np.argmax(criteria_weights)]}**方面表现突出（该因素权重达 {max(criteria_weights):.1%}）"))
display(Markdown(f"- **风险提示**：{best_supplier}在{'价格' if best_supplier == 'C公司' else '质量'}方面相对较弱，建议签订价格保护协议"))

# %%
# %% [markdown]
# ### 2. 生产计划建议

# %%
# 生产计划建议
profit_x = x_opt * profit[0]
profit_y = y_opt * profit[1]
profit_ratio_x = profit_x / max_profit
profit_ratio_y = profit_y / max_profit

display(Markdown(f"### 📈 生产计划建议"))
display(Markdown(f"**最优生产方案**：智能空调 {x_opt:.0f} 台 + 智能冰箱 {y_opt:.0f} 台"))
display(Markdown(f"- **预期总利润**：{max_profit:,.0f} 元"))
display(Markdown(f"  - 智能空调贡献：{profit_x:,.0f} 元 ({profit_ratio_x:.1%})"))
display(Markdown(f"  - 智能冰箱贡献：{profit_y:,.0f} 元 ({profit_ratio_y:.1%})"))
display(Markdown(f"- **资源利用**：原材料 {material_usage[0]*x_opt+material_usage[1]*y_opt:.0f}/{material_constraint} 单位，工时 {labor_usage[0]*x_opt+labor_usage[1]*y_opt:.0f}/{labor_constraint} 小时"))

# %%
# %% [markdown]
# ### 3. 优化建议与敏感性分析

# %%
# 敏感性分析（影子价格）
# 原材料约束的影子价格
material_shadow_price = (max_profit - (-linprog(c, A_ub=A, b_ub=[material_constraint-1, labor_constraint, -min_x, -min_y, max_x, max_y]).fun)) 
# 工时约束的影子价格
labor_shadow_price = (max_profit - (-linprog(c, A_ub=A, b_ub=[material_constraint, labor_constraint-1, -min_x, -min_y, max_x, max_y]).fun))

display(Markdown("### 🔍 优化建议与敏感性分析"))
display(Markdown(f"- **原材料影子价格**：{material_shadow_price:.0f} 元/单位 - 每增加1单位原材料，利润可增加约 {material_shadow_price:.0f} 元"))
display(Markdown(f"- **工时影子价格**：{labor_shadow_price:.0f} 元/小时 - 每增加1工时，利润可增加约 {labor_shadow_price:.0f} 元"))
display(Markdown("- **优先优化方向**：原材料供应（影子价格更高，优化潜力更大）"))
display(Markdown(f"- **安全边际**：X产品可减少 {x_opt - min_x:.0f} 台，Y产品可减少 {y_opt - min_y:.0f} 台而不影响最优解"))

# %%
# %% [markdown]
# ### 4. 最终业务建议

# %%
display(Markdown("## 💼 最终业务建议"))

# 业务建议
business_recommendations = [
    f"1. **立即与{best_supplier}签订采购合同**，重点关注价格保护条款（因其价格竞争力较弱）",
    "2. 按照最优生产计划组织生产：80台空调 + 93台冰箱",
    "3. 优先拓展原材料供应渠道，每增加1单位原材料可多获利润200元",
    "4. 评估增加工时产能的可行性（当前工时已满负荷）",
    "5. 考虑适当降低冰箱的最低需求（当前93台远超40台底线），释放更多产能给高利润产品",
    "6. 每季度重新评估供应商表现，重点关注质量指标的变化"
]

for rec in business_recommendations:
    display(Markdown(f"- {rec}"))

# %%
# %% [markdown]
# ## 附录：模型验证与扩展思考

# %%
# %% [markdown]
# ### 1. 模型验证

# %%
display(Markdown("### 模型验证"))

# 验证1：改变原材料限制，检查结果是否合理
test_material = material_constraint + 20
test_res = linprog(c, A_ub=A, b_ub=[test_material, labor_constraint, -min_x, -min_y, max_x, max_y])
test_profit = -test_res.fun if test_res.success else max_profit

display(Markdown(f"- **验证1**：原材料增加20单位，利润从 {max_profit:,.0f} 元增至 {test_profit:,.0f} 元（+{test_profit-max_profit:,.0f}元），符合影子价格预期"))
display(Markdown(f"- **验证2**：若原材料减少20单位，最优解将变为 ({linprog(c, A_ub=A, b_ub=[material_constraint-20, labor_constraint, -min_x, -min_y, max_x, max_y]).x[0]:.0f}, {linprog(c, A_ub=A, b_ub=[material_constraint-20, labor_constraint, -min_x, -min_y, max_x, max_y]).x[1]:.0f})，利润降至 {-linprog(c, A_ub=A, b_ub=[material_constraint-20, labor_constraint, -min_x, -min_y, max_x, max_y]).fun:,.0f} 元"))

# %%
# %% [markdown]
# ### 2. 扩展思考

# %%
display(Markdown("### 扩展思考"))

# 扩展点
extensions = [
    "• **多目标优化**：除了利润，还可以考虑市场份额、客户满意度等目标",
    "• **不确定性分析**：原材料价格波动、市场需求变化等风险因素",
    "• **整数规划**：如果产品必须整数生产，需要使用整数规划(MILP)",
    "• **动态规划**：考虑多期决策，平衡短期利润和长期发展",
    "• **供应商组合**：不局限于单一供应商，可考虑多供应商组合策略"
]

for ext in extensions:
    display(Markdown(f"- {ext}"))

# %%
# %% [markdown]
# ## 总结
# 
# 本案例完整展示了如何结合**层次分析法(AHP)**和**线性规划**解决企业决策问题：
# 
# 1. **AHP部分**：科学评估3家供应商，选择{best_supplier}（综合得分{best_score:.3f}）
# 2. **线性规划部分**：基于选定供应商，制定最优生产计划（{x_opt:.0f}台空调 + {y_opt:.0f}台冰箱，利润{max_profit:,.0f}元）
# 3. **综合建议**：提供可执行的业务建议，包括供应商管理、生产优化和风险控制
# 
# **核心价值**：将主观判断(AHP)与定量优化(线性规划)结合，形成完整的决策闭环，为企业提供数据驱动的决策支持。